In [19]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import surprise

data = surprise.Dataset.load_builtin('ml-100k')

df = pd.DataFrame(data.raw_ratings, columns = ['userId', 'item', 'rating', 'id'])
df.head()

cnt = Counter(df['userId']) # 각 userId가 몇 번 등장했는지 count한 뒤에
idx_sorted = sorted(list(cnt.keys()), key=lambda x: -cnt[x]) # 빈도가 높은 순서대로 userId를 정렬

idx = np.zeros(len(df)) # df의 전체 row 길이에 해당하는 vector를 만든 뒤에
for i in idx_sorted[:400]:
    idx += (df['userId'] == i)
    print(idx)
    # 빈도 400등까지의 userId에 해당하는 row에 1씩을 더해준 뒤,
idx = np.array(idx, dtype=bool) # Boolean 타입으로 바꾸어 줌. -> Bool Indexing을 위한 밑작업

new_df = df[idx].iloc[:,:3] # 위의 자료를 이용해 인덱싱을 한 뒤 새로운 df를 만드는데, 마지막 열(timestamp)은 날림

from sklearn.model_selection import train_test_split
train, test = train_test_split(new_df, test_size=0.3, random_state=42) # 3:7 비율로 test-train split

utility_matrix = train.pivot(index='userId', columns='item', values='rating').fillna(0) 

def cos_sim(x, y): # cosine 유사도를 return하는 함수를 define
    return np.dot(x, y) ** 2 / (np.dot(x, x) * np.dot(y, y))

def find_neighbors(utility_matrix, k, sim_fun): # 주어진 유사도를 기준으로 k개의 neighbor를 구해보자!
    sim_dict = {} # 각 user들의 이웃들을 내포한 딕셔너리로 만들 것임!
    cnt = 0 # 밑에서 반복문 몇 번 돌았는지 확인하려고...
    for i in utility_matrix.index: # utility_matrix의 index들 = userId들
        cnt += 1 # 반복 1회당 카운트 한 번씩 올려주고
        ranking = [(0,0)] * k
        for j in utility_matrix.index.drop(i): # i를 제외한 나머지 userId에 대해 반복
            ranking.append((j, sim_fun(utility_matrix.loc[i], utility_matrix.loc[j]))) # (j, simmilarity between i & j) append
            ranking = sorted(ranking, key=lambda x: -x[1])[:k] # 각 튜플의 뒤의 원소를 기준으로 내림차순한 뒤(유클리디언은 오름차순 해야됨), k개까지만 cut
        sim_dict[i] = ranking # 위에서 만든 리스트를 sim_dict의 key = i에 대해 append함
        if not cnt%10: print(cnt) # 10단위로 카운트 수를 출력... 얼마나 진행되고 있는지 시각적으로 확인코자 하는 수단
    return sim_dict

neighbors = find_neighbors(utility_matrix, 10, cos_sim)

def prediction(user, movie, neighbors, utility_matrix): # 이제 neighbor들의 평균을 냄으로써 평점을 예측해봅시다
    my_neighbors = neighbors[user] # 지정된 user의 이웃들을 꺼낸 뒤
    rates = []
    for i, _ in my_neighbors:
        r = utility_matrix.loc[i, movie] # 이웃의 해당 영화에 대한 평점을 꺼내옵니다
        if r > 0: rates.append(r) # 정보가 있을 경우에 리스트에 append
    if rates == []: return None # 만약 리스트가 비어있다면, None을 반환 (왜냐면, 이웃 중에 그 영화를 본 사람이 없는 것이므로)
    else: return sum(rates)/len(rates) # 아니라면 평균을 return합시다

def testing(test, neighbors, utility_matrix): # test data에 대해 test하는 함수
    predicted = []
    movies = utility_matrix.columns # train data에 있는 영화 목록
    for i in range(len(test)): # test set에 대해 반복을 돌립니다
        movie = test.iloc[i,0]; user = test.iloc[i,1] # 각 row의 movie, user Id를 따옴
        if (user not in neighbors) or (movie not in movies): # 만약 해당 user의 neighbor data가 없거나, movie가 train movie 목록에 없다면
            predicted.append(None) # 그 데이터에 관해서는 test를 할 수 없음
        else: predicted.append(prediction(user, movie, neighbors, utility_matrix)) # 아니라면 예측을 합니다
    return predicted

test['prediction'] = testing(test, neighbors, utility_matrix)
test['diff'] = test['rating'] - test['prediction']
test['diff'].apply(abs).mean()

from sklearn.model_selection import KFold  
cv = KFold(n_splits=5, shuffle=True, random_state=0) 
for train_index, test_index in cv.split(new_df): 
    train = new_df.iloc[train_index,] 
    test = new_df.iloc[test_index,] 
    test['prediction'] = testing(test, neighbors, utility_matrix) 
    test['diff'] = test['rating'] - test['prediction'] 
    
     
    print(test['diff'].apply(abs).mean()) 


# SELECT *, quantity*price AS stock_price FROM quest1 WHERE price > 10000 OR quantity*price > 200000;
# SELECT name,YEAR(CURRENT_TIMESTAMP())-YEAR(birthday)+1 AS age, birthday, CASE WHEN YEAR(birthday) < 2000 THEN 'yes' ELSE 'no' END AS "adult" FROM quest2;

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
99970    0.0
99971    0.0
99972    0.0
99973    0.0
99974    0.0
99975    0.0
99976    0.0
99977    0.0
99978    0.0
99979    0.0
99980    0.0
99981    0.0
99982    0.0
99983    0.0
99984    0.0
99985    0.0
99986    0.0
99987    0.0
99988    0.0
99989    0.0
99990    0.0
99991    0.0
99992    0.0
99993    0.0
99994    0.0
99995    0.0
99996    0.0
99997    0.0
99998    0.0
99999    0.0
Name: userId, Length: 100000, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12    

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       1.0
16       0.0
17       0.0
18       0.0
19       1.0
20       0.0
21       0.0
22       0.0
23       0.0
24       1.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
99970    0.0
99971    0.0
99972    0.0
99973    0.0
99974    0.0
99975    1.0
99976    0.0
99977    0.0
99978    0.0
99979    0.0
99980    0.0
99981    0.0
99982    1.0
99983    0.0
99984    0.0
99985    0.0
99986    0.0
99987    0.0
99988    0.0
99989    0.0
99990    0.0
99991    0.0
99992    0.0
99993    0.0
99994    0.0
99995    0.0
99996    0.0
99997    1.0
99998    1.0
99999    0.0
Name: userId, Length: 100000, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12    

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       1.0
16       0.0
17       0.0
18       0.0
19       1.0
20       0.0
21       0.0
22       0.0
23       0.0
24       1.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
99970    0.0
99971    0.0
99972    0.0
99973    0.0
99974    0.0
99975    1.0
99976    0.0
99977    0.0
99978    0.0
99979    0.0
99980    0.0
99981    0.0
99982    1.0
99983    0.0
99984    0.0
99985    0.0
99986    0.0
99987    0.0
99988    0.0
99989    0.0
99990    0.0
99991    0.0
99992    0.0
99993    0.0
99994    1.0
99995    1.0
99996    0.0
99997    1.0
99998    1.0
99999    0.0
Name: userId, Length: 100000, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12    

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       1.0
16       0.0
17       1.0
18       0.0
19       1.0
20       0.0
21       0.0
22       0.0
23       0.0
24       1.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
99970    0.0
99971    0.0
99972    0.0
99973    0.0
99974    0.0
99975    1.0
99976    0.0
99977    0.0
99978    0.0
99979    0.0
99980    0.0
99981    0.0
99982    1.0
99983    0.0
99984    0.0
99985    0.0
99986    0.0
99987    0.0
99988    0.0
99989    0.0
99990    0.0
99991    0.0
99992    0.0
99993    0.0
99994    1.0
99995    1.0
99996    0.0
99997    1.0
99998    1.0
99999    0.0
Name: userId, Length: 100000, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12    

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       1.0
12       0.0
13       0.0
14       0.0
15       1.0
16       0.0
17       1.0
18       1.0
19       1.0
20       0.0
21       0.0
22       1.0
23       1.0
24       1.0
25       1.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
99970    0.0
99971    0.0
99972    0.0
99973    0.0
99974    0.0
99975    1.0
99976    0.0
99977    0.0
99978    0.0
99979    0.0
99980    1.0
99981    0.0
99982    1.0
99983    0.0
99984    0.0
99985    0.0
99986    0.0
99987    0.0
99988    0.0
99989    0.0
99990    0.0
99991    0.0
99992    0.0
99993    0.0
99994    1.0
99995    1.0
99996    0.0
99997    1.0
99998    1.0
99999    0.0
Name: userId, Length: 100000, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       1.0
12    

...

0        0.0
1        0.0
2        1.0
3        1.0
4        0.0
5        1.0
6        0.0
7        0.0
8        1.0
9        1.0
10       1.0
11       1.0
12       1.0
13       1.0
14       1.0
15       1.0
16       0.0
17       1.0
18       1.0
19       1.0
20       1.0
21       0.0
22       1.0
23       1.0
24       1.0
25       1.0
26       1.0
27       1.0
28       0.0
29       1.0
        ... 
99970    0.0
99971    1.0
99972    1.0
99973    0.0
99974    1.0
99975    1.0
99976    1.0
99977    1.0
99978    0.0
99979    1.0
99980    1.0
99981    0.0
99982    1.0
99983    0.0
99984    1.0
99985    1.0
99986    1.0
99987    1.0
99988    0.0
99989    1.0
99990    1.0
99991    0.0
99992    1.0
99993    1.0
99994    1.0
99995    1.0
99996    1.0
99997    1.0
99998    1.0
99999    0.0
Name: userId, Length: 100000, dtype: float64
0        0.0
1        0.0
2        1.0
3        1.0
4        0.0
5        1.0
6        0.0
7        0.0
8        1.0
9        1.0
10       1.0
11       1.0
12    

0        0.0
1        0.0
2        1.0
3        1.0
4        0.0
5        1.0
6        0.0
7        0.0
8        1.0
9        1.0
10       1.0
11       1.0
12       1.0
13       1.0
14       1.0
15       1.0
16       0.0
17       1.0
18       1.0
19       1.0
20       1.0
21       0.0
22       1.0
23       1.0
24       1.0
25       1.0
26       1.0
27       1.0
28       0.0
29       1.0
        ... 
99970    0.0
99971    1.0
99972    1.0
99973    0.0
99974    1.0
99975    1.0
99976    1.0
99977    1.0
99978    0.0
99979    1.0
99980    1.0
99981    0.0
99982    1.0
99983    0.0
99984    1.0
99985    1.0
99986    1.0
99987    1.0
99988    0.0
99989    1.0
99990    1.0
99991    0.0
99992    1.0
99993    1.0
99994    1.0
99995    1.0
99996    1.0
99997    1.0
99998    1.0
99999    0.0
Name: userId, Length: 100000, dtype: float64
0        0.0
1        0.0
2        1.0
3        1.0
4        0.0
5        1.0
6        0.0
7        0.0
8        1.0
9        1.0
10       1.0
11       1.0
12    

0        0.0
1        0.0
2        1.0
3        1.0
4        0.0
5        1.0
6        0.0
7        1.0
8        1.0
9        1.0
10       1.0
11       1.0
12       1.0
13       1.0
14       1.0
15       1.0
16       0.0
17       1.0
18       1.0
19       1.0
20       1.0
21       0.0
22       1.0
23       1.0
24       1.0
25       1.0
26       1.0
27       1.0
28       0.0
29       1.0
        ... 
99970    0.0
99971    1.0
99972    1.0
99973    0.0
99974    1.0
99975    1.0
99976    1.0
99977    1.0
99978    0.0
99979    1.0
99980    1.0
99981    0.0
99982    1.0
99983    0.0
99984    1.0
99985    1.0
99986    1.0
99987    1.0
99988    0.0
99989    1.0
99990    1.0
99991    0.0
99992    1.0
99993    1.0
99994    1.0
99995    1.0
99996    1.0
99997    1.0
99998    1.0
99999    0.0
Name: userId, Length: 100000, dtype: float64
0        0.0
1        0.0
2        1.0
3        1.0
4        0.0
5        1.0
6        0.0
7        1.0
8        1.0
9        1.0
10       1.0
11       1.0
12    

0        0.0
1        1.0
2        1.0
3        1.0
4        0.0
5        1.0
6        1.0
7        1.0
8        1.0
9        1.0
10       1.0
11       1.0
12       1.0
13       1.0
14       1.0
15       1.0
16       0.0
17       1.0
18       1.0
19       1.0
20       1.0
21       0.0
22       1.0
23       1.0
24       1.0
25       1.0
26       1.0
27       1.0
28       1.0
29       1.0
        ... 
99970    0.0
99971    1.0
99972    1.0
99973    0.0
99974    1.0
99975    1.0
99976    1.0
99977    1.0
99978    0.0
99979    1.0
99980    1.0
99981    0.0
99982    1.0
99983    0.0
99984    1.0
99985    1.0
99986    1.0
99987    1.0
99988    0.0
99989    1.0
99990    1.0
99991    0.0
99992    1.0
99993    1.0
99994    1.0
99995    1.0
99996    1.0
99997    1.0
99998    1.0
99999    0.0
Name: userId, Length: 100000, dtype: float64
0        0.0
1        1.0
2        1.0
3        1.0
4        0.0
5        1.0
6        1.0
7        1.0
8        1.0
9        1.0
10       1.0
11       1.0
12    

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
